## Notebook 4: Mapping a Flood with Radar

**Radar can see through clouds and at night - perfect for disaster response.**

When floods happen, clouds often block optical satellites right when we need imagery most. Radar solves this problem. Synthetic Aperture Radar (SAR) actively sends microwave pulses and measures what bounces back - it works day or night, rain or shine.

In this notebook, we'll:

- Use Sentinel-1 SAR data to map the **March 2019 Missouri River flood**
- Compare before and after images to detect flooded areas
- Create a flood extent map using change detection

The 2019 Missouri River flood was catastrophic - a "bomb cyclone" in mid-March caused record flooding across Nebraska, Iowa, and Missouri, causing over $3 billion in damages.

## Setup

Same initialization as previous notebooks.

In [1]:
%pip install -q geemap folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
from google.cloud import storage

# Initialize Earth Engine
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"
ee.Initialize(project=PROJECT)

import geemap.foliumap as geemap

print("Ready!")

Ready!


## Why SAR for flood mapping?

**SAR (Synthetic Aperture Radar)** is fundamentally different from optical imagery:

| Optical (Landsat, Sentinel-2) | Radar (Sentinel-1) |
|-------------------------------|--------------------|
| Measures reflected sunlight | Sends its own microwave pulses |
| Blocked by clouds | Sees through clouds |
| Needs daylight | Works day and night |
| Shows colors/reflectance | Shows surface texture/roughness |

**Why water appears dark in SAR:**
- Calm water acts like a mirror for radar waves
- The signal bounces away from the satellite (specular reflection)
- Very little energy returns = dark pixels
- Rough surfaces (land, vegetation) scatter energy back = bright pixels

**Key settings for flood mapping:**
- **VV polarization**: Vertical send, vertical receive - best for water detection
- **Consistent orbit**: Use only ascending OR descending passes to compare apples-to-apples

## Define area of interest

We'll focus on the Missouri River floodplain near the Nebraska-Iowa-Missouri border, where flooding was severe in March 2019. This area includes parts of Holt and Atchison counties in Missouri.

In [3]:
# Define area of interest - Missouri River floodplain near I-29 corridor
# This region saw catastrophic flooding in March 2019
aoi = ee.Geometry.Rectangle([-95.8, 39.8, -95.2, 40.3])

# Center point for map
center_lat = 40.05
center_lon = -95.5

print("Area of interest: Missouri River floodplain near Mound City, MO")

Area of interest: Missouri River floodplain near Mound City, MO


## Load Sentinel-1 SAR imagery

Sentinel-1 provides C-band SAR data globally. We'll use the Ground Range Detected (GRD) product, which is already processed and terrain-corrected.

**Timeline of 2019 Missouri River flood:**
- March 13-14: "Bomb cyclone" hits the region
- March 15-17: Levee breaches, catastrophic flooding begins
- March 18+: Flood waters spread across floodplain

We'll compare:
- **Before**: February 1-22, 2019 (pre-flood conditions)
- **After**: March 18 - April 5, 2019 (peak flooding)

In [4]:
# Sentinel-1 GRD collection
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")

# Filter parameters for consistent comparison
def get_s1_composite(start_date, end_date, geometry):
    """Get Sentinel-1 VV composite for a date range."""
    return (
        s1.filterBounds(geometry)
          .filterDate(start_date, end_date)
          .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide swath
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
          .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))  # Use ascending for this region
          .select('VV')
          .median()
          .clip(geometry)
    )

# Before flood: February 2019
before = get_s1_composite('2019-02-01', '2019-02-22', aoi)

# During flood: Late March 2019
after = get_s1_composite('2019-03-18', '2019-04-05', aoi)

print("Loaded before and after Sentinel-1 composites")

Loaded before and after Sentinel-1 composites


## Visualize before and after

SAR backscatter is measured in decibels (dB). Typical values:
- **Water**: -20 to -25 dB (very dark)
- **Bare soil**: -10 to -15 dB (medium gray)
- **Vegetation/urban**: -5 to 0 dB (bright)

Look for areas that become **much darker** in the "after" image - these are flooded.

In [5]:
# SAR visualization parameters (grayscale, dB values)
sar_vis = {
    'min': -25,
    'max': 0,
    'palette': ['black', 'white']
}

# Create split map to compare before/after
left_layer = geemap.ee_tile_layer(before, sar_vis, 'Before (Feb 2019)')
right_layer = geemap.ee_tile_layer(after, sar_vis, 'After (Mar-Apr 2019)')

m = geemap.Map(center=[center_lat, center_lon], zoom=10)
m.split_map(left_layer, right_layer)
m

## Create flood extent map

We'll use **change detection** to identify flooded areas:

1. Calculate the difference: `before - after`
2. Where the difference is **positive** (before was brighter than after), flooding occurred
3. Apply a threshold to create a binary flood map

A threshold of ~3 dB is commonly used - this represents a significant decrease in backscatter.

In [6]:
# Calculate difference (before - after)
# Positive values = backscatter decreased = likely flooding
difference = before.subtract(after)

# Threshold to identify flood pixels
# Where backscatter dropped by more than 3 dB, likely flooded
flood_threshold = 3  # dB
flood_mask = difference.gt(flood_threshold)

print(f"Flood detection threshold: {flood_threshold} dB decrease in backscatter")

Flood detection threshold: 3 dB decrease in backscatter


In [7]:
# Create comprehensive flood map
m2 = geemap.Map(center=[center_lat, center_lon], zoom=11)

# Add the after image as base (shows current conditions)
m2.addLayer(after, sar_vis, 'SAR After Flood (Mar-Apr 2019)')

# Add the difference image
diff_vis = {
    'min': -5,
    'max': 10,
    'palette': ['blue', 'white', 'red']  # blue=wetter after, red=drier after (flooded)
}
m2.addLayer(difference, diff_vis, 'Difference (Before - After)')

# Add flood extent in blue
flood_vis = {'palette': ['0000FF']}  # Blue for flood
m2.addLayer(flood_mask.selfMask(), flood_vis, 'Flood Extent')

# Add AOI boundary
m2.addLayer(aoi, {'color': 'yellow'}, 'Area of Interest')

m2

## What do you notice?

Toggle the layers on/off in the layer control (top right) to explore:

- **SAR After Flood**: Dark areas are standing water. Notice how much of the floodplain is dark.
- **Difference**: Red areas show where backscatter dropped significantly (flooding). Blue areas got brighter (unusual - possibly debris or rough water).
- **Flood Extent**: Blue overlay shows our detected flood pixels.

The Missouri River channel is always dark (permanent water). What we're detecting is the **expansion** of water into normally dry areas.

## Calculate flooded area

Let's quantify how much land was flooded in our study area.

In [8]:
# Calculate flooded area in square kilometers
flood_area = flood_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=30,  # 30m resolution
    maxPixels=1e9
)

# Convert from sq meters to sq km
area_sqkm = ee.Number(flood_area.get('VV')).divide(1e6)

# Also calculate total AOI area for context
total_area = aoi.area().divide(1e6)

print(f"Flooded area: {area_sqkm.getInfo():.1f} sq km")
print(f"Total study area: {total_area.getInfo():.1f} sq km")
print(f"Percent flooded: {(area_sqkm.divide(total_area).multiply(100)).getInfo():.1f}%")

Flooded area: 345.7 sq km
Total study area: 2839.4 sq km
Percent flooded: 12.2%


Percent flooded: 12.2%


## Try it yourself

Ideas to explore:

1. **Adjust the threshold**: Try values from 2-5 dB. Lower values detect more flooding but may include false positives. What looks most accurate?

2. **Try VH polarization**: Change `'VV'` to `'VH'` (vertical send, horizontal receive). VH is more sensitive to vegetation - how does the flood map change?

3. **Different flood events**: 
   - Hurricane Harvey (Houston, Aug 2017)
   - Mississippi River 2011
   - Your own area of interest

4. **Add optical imagery**: Load a cloud-free Sentinel-2 image from before the flood to see what the landscape normally looks like.

5. **Time series**: Create a series of flood maps over several weeks to see how flooding evolved and receded.

6. **Combine with elevation**: Use a DEM to mask out areas too high to flood - this reduces false positives on hillsides.